<a href="https://colab.research.google.com/github/Sharanya-Parimanoharan/AI-Generated-Text-Detection/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.utils.rnn as rnn_utils
import random
from sklearn.metrics import roc_curve, roc_auc_score,confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
       torch.cuda.manual_seed_all(seed)



In [ ]:
# Check if GPU is available
is_cuda = torch.cuda.is_available()
##
##
###is_cuda = torch.cuda.is_available()
##
### If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

In [ ]:
df = pd.read_csv("drive/MyDrive/data_2.csv")
print(df)
df.head()

In [ ]:

X,y = df['text'].values,df['label'].values
##startify is set to y to ensure that distribution of target variable is the same in the training and testing datasets
x_train,x_test,y_train,y_test = train_test_split(X,y,stratify=y)
x=x_test.copy()
y_t=y_test.copy()
print(f'shape of train data is {x_train.shape}')
print(f'shape of test data is {x_test.shape}')


In [ ]:

def preprocess_string(s):
    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespaces with no space
    s = re.sub(r"\s+", '', s)
    # replace digits with no space
    s = re.sub(r"\d", '', s)

    return s

def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        try:
            if len(review) != 0:
                features[ii, -len(review):] = np.array(review)[:seq_len]
        except ValueError as e:
           # print(f"Error processing review at index {ii}: {e}")
           # print(f"Problematic review data: {review}")
            # Handle the error gracefully, e.g., by skipping this data point
            continue
    return features
print(y_train)


In [ ]:

def tockenize(x_train,y_train,x_val,y_val):
    word_list = []


    ##loop over each sentence in X_train and then loop over each word
    for sent in x_train:
        for word in sent.lower().split():
            word = preprocess_string(word)
            #if word not in stop_words and word != '':
            word_list.append(word)

    #Counter object of word_list is created, this object counts the frequency of word in word_list
    corpus = Counter(word_list)
    # sorting on the basis of most common words(by frequency) and select top 1000
    corpus_ = sorted(corpus,key=corpus.get,reverse=True)[:1000]


    # creating a dict-maps each word in corpus_ to an integer
    onehot_dict = {w:i+1 for i,w in enumerate(corpus_)}

    # tockenize
    final_list_train,final_list_test = [],[]
    #words integer value is added to the final_list_train
    for sent in x_train:
            final_list_train.append([onehot_dict[preprocess_string(word)] for word in sent.lower().split()
                                     if preprocess_string(word) in onehot_dict.keys()])
    for sent in x_val:
            final_list_test.append([onehot_dict[preprocess_string(word)] for word in sent.lower().split()
                                    if preprocess_string(word) in onehot_dict.keys()])

    #we have very less number of reviews with length > 500.
    #So we will consideronly those below it.
    final_list_train = padding_(final_list_train,500)
    final_list_test = padding_(final_list_test,500)

    encoded_train = [1 if label =='"AI"' else 0 for label in y_train]
    encoded_test = [1 if label =='"AI"' else 0 for label in y_val]
    return np.array(final_list_train), np.array(encoded_train),np.array(final_list_test), np.array(encoded_test),onehot_dict

x_train_pad,y_train,x_test_pad,y_test,vocab = tockenize(x_train,y_train,x_test,y_test)
print(f'Length of vocabulary is {len(vocab)}')
y__t=y_test.copy()
print(y__t)


In [ ]:

# Function to generate bigrams from text and count their frequencies
def generate_bigrams(text_data):
    bigram_data = []
    bigram_counter = Counter()  # Initialize a Counter to count bigram frequencies
    for text in text_data:
        tokens = text.split()
        bigrams = [f"{tokens[i]} {tokens[i+1]}" for i in range(len(tokens) - 1)]
        bigram_data.append(bigrams)
        bigram_counter.update(bigrams)  # Update the bigram frequencies
    return bigram_data, bigram_counter

# Generate bigrams and count frequencies for training and testing data
x_train_bigrams, bigram_counter_train = generate_bigrams(x_train)
x_test_bigrams, bigram_counter_test = generate_bigrams(x_test)

# Create a vocabulary for bigrams
bigram_vocab = {bigram: i + len(vocab) + 1 for i, bigram in enumerate(bigram_counter_train.keys())}


x_train_bigrams_pad = padding_(x_train_bigrams, 500)
x_test_bigrams_pad = padding_(x_test_bigrams, 500)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Assuming you have labels (y_train and y_test)

# Extract indices for 'AI' and 'Human' labels in the training set
indices_ai_train = [i for i, label in enumerate(y_train) if label == 1]
indices_human_train = [i for i, label in enumerate(y_train) if label == 0]

# Extract indices for 'AI' and 'Human' labels in the test set
indices_ai_test = [i for i, label in enumerate(y_test) if label == 1]
indices_human_test = [i for i, label in enumerate(y_test) if label == 0]

# Count frequencies of bigrams for 'AI' and 'Human' in the training set
bigram_counts_ai_train = Counter([bigram for sublist in x_train_bigrams_pad[indices_ai_train] for bigram in sublist])
bigram_counts_human_train = Counter([bigram for sublist in x_train_bigrams_pad[indices_human_train] for bigram in sublist])

# Count frequencies of bigrams for 'AI' and 'Human' in the test set
bigram_counts_ai_test = Counter([bigram for sublist in x_test_bigrams_pad[indices_ai_test] for bigram in sublist])
bigram_counts_human_test = Counter([bigram for sublist in x_test_bigrams_pad[indices_human_test] for bigram in sublist])

# Plot bigram distribution for 'AI' and 'Human' in the training set
bar_width = 0.35
r1 = np.arange(len(bigram_counts_ai_train))
r2 = [x + bar_width for x in r1]

plt.figure(figsize=(15, 6))
plt.bar(r1, bigram_counts_ai_train.values(), color='blue', width=bar_width, edgecolor='grey', label='AI - Train')
plt.bar(r2, bigram_counts_human_train.values(), color='orange', width=bar_width, edgecolor='grey', label='Human - Train')

plt.title('Bigram Distribution - Training Set')
plt.xlabel('Bigrams')
plt.ylabel('Frequency')
plt.xticks([r + bar_width/2 for r in range(len(bigram_counts_ai_train))], bigram_counts_ai_train.keys(), rotation=45, ha='right')
plt.legend()
plt.show()

# Plot bigram distribution for 'AI' and 'Human' in the test set
r1 = np.arange(len(bigram_counts_ai_test))
r2 = [x + bar_width for x in r1]

plt.figure(figsize=(15, 6))
plt.bar(r1, bigram_counts_ai_test.values(), color='blue', width=bar_width, edgecolor='grey', label='AI - Test')
plt.bar(r2, bigram_counts_human_test.values(), color='orange', width=bar_width, edgecolor='grey', label='Human - Test')

plt.title('Bigram Distribution - Test Set')
plt.xlabel('Bigrams')
plt.ylabel('Frequency')
plt.xticks([r + bar_width/2 for r in range(len(bigram_counts_ai_test))], bigram_counts_ai_test.keys(), rotation=45, ha='right')
plt.legend()
plt.show()


In [ ]:

# create Tensor datasets from pytorch library
#torch.from_numpy()- converts numpy array to  pytorch tensors
train_data = TensorDataset(torch.from_numpy(x_train_pad), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(x_test_pad), torch.from_numpy(y_test))

# dataloaders
batch_size = 25

#Creates dataloader from pytorch library
##data loader class used to load in to batches while training
# make sure to SHUFFLE your data before each epoch
train_loader = DataLoader(train_data, shuffle=False , batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=False, batch_size=batch_size)


# obtain one batch of training data
dataiter = iter(train_loader)
# sample_x and sample_y - input and otput data from the first batch of training
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample input: \n', sample_y)


In [ ]:
class TextClassificationRNN(nn.Module):
    def __init__(self,no_layers,vocab_size,bigram_vocab_size,hidden_dim,embedding_dim,drop_prob=0.5):
        super(TextClassificationRNN,self).__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim


        self.no_layers = no_layers
        self.vocab_size = vocab_size
        self.bigram_vocab_size = bigram_vocab_size

        # embedding and LSTM layers    converts tokenize words into dense vectors
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        #initializes weights and biases internally
        #lstm
        #self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
                           #num_layers=no_layers,batch_first=True)

        # LSTM layers
        self.lstm1 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=1, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=1, batch_first=True)

        # Embedding and LSTM layers for n-grams
        self.bigram_embedding = nn.Embedding(bigram_vocab_size, embedding_dim)
        self.bigram_lstm1 = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=1, batch_first=True)
        self.bigram_lstm2 = nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=1, batch_first=True)



        # Custom weight initialization
        self.init_weights()

        # dropout layer
        self.dropout = nn.Dropout(0.3)

        # linear and sigmoid layer  -linear layers transforms lstm hidden states
        self.fc = nn.Linear(2*self.hidden_dim, output_dim)
        self.sig = nn.Sigmoid()

##        self.softmax = nn.Softmax(dim=1)



    def init_weights(self):
        for name, param in self.lstm1.named_parameters():
            if 'weight' in name:
                nn.init.xavier_normal_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)

        for name, param in self.lstm2.named_parameters():
            if 'weight' in name:
                nn.init.xavier_normal_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)


    def forward(self,x,hidden):
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)  # shape: B x S x Feature   since batch = True
        lstm_out1, _ = self.lstm1(embeds)
        # LSTM layer 2
        lstm_out2, _ = self.lstm2(lstm_out1)
        # Flatten LSTM output
        lstm_out2 = lstm_out2.contiguous().view(-1, self.hidden_dim)


        bigram_embeds = self.bigram_embedding(x)
        bigram_lstm_out1, _ = self.bigram_lstm1(bigram_embeds)
        bigram_lstm_out2, _ = self.bigram_lstm2(bigram_lstm_out1)
        bigram_lstm_out2 = bigram_lstm_out2.contiguous().view(-1, self.hidden_dim)

        # Concatenate unigram and bigram representations
        combined_representation = torch.cat((lstm_out2, bigram_lstm_out2), dim=1)

        #print(embeds.shape)  #[50, 500, 1000]
        #lstm_out, hidden = self.lstm(embeds, hidden)

        #lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully connected layer
       # out = self.dropout(lstm_out2)
        out = self.dropout(combined_representation)

        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)
      #  sig_out = nn.functional.softmax(out, dim=1)


        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)

        sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return sig_out, hidden

##        # softmax function
##      #  softmax_out = F.softmax(out,dim=1)
##
##    #    # reshape to be batch_size first
##        out = out.view(batch_size, -1)
##        out = out[:, -1]  # get last batch of labels
##
##        # return last softmax output and hidden state
##        return out, hidden


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        h0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        c0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden

In [ ]:
no_layers = 2
vocab_size = len(vocab)+1 #extra 1 for padding
bigram_vocab_size= len(bigram_vocab) +1
embedding_dim = 64
output_dim = 2
hidden_dim = 256


model = TextClassificationRNN(no_layers,vocab_size,bigram_vocab_size,hidden_dim,embedding_dim,drop_prob=0.5)


In [ ]:


#moving to gpu
model.to(device)

print(model)

# loss and optimization functions

#learning rate
lr=0.0001

#loss function used for binary classification probelm
##criterion = nn.BCELoss()
criterion=nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [ ]:
# function to predict accuracy
#takes predicted value and true label, rounds off predicted value to nearest integer and compares it
def acc(pred,label):
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label.squeeze()).item()

In [ ]:
clip = 5
epochs = 5
valid_loss_min = np.Inf
# train for some number of epochs
epoch_tr_loss,epoch_vl_loss = [],[]
epoch_tr_acc,epoch_vl_acc = [],[]

for epoch in range(epochs):
    train_losses = []
    train_acc = 0.0
    model.train()
    # initialize hidden state
    h = model.init_hidden(batch_size)

    for inputs, labels in train_loader:

        inputs, labels = inputs.to(device), labels.to(device)


        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        model.zero_grad()
        output,h = model(inputs,h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        train_losses.append(loss.item())


        # calculating accuracy
        accuracy = acc(output,labels)
        train_acc += accuracy
        #`clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()



    val_h = model.init_hidden(batch_size)
    val_losses = []
    val_acc = 0.0
    model.eval()
    for inputs, labels in valid_loader:
            val_h = tuple([each.data for each in val_h])

            inputs, labels = inputs.to(device), labels.to(device)

            output, val_h = model(inputs, val_h)
            val_loss = criterion(output.squeeze(), labels.float())

            val_losses.append(val_loss.item())

            accuracy = acc(output,labels)
            val_acc += accuracy

    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)
    epoch_train_acc = train_acc/len(train_loader.dataset)
    epoch_val_acc = val_acc/len(valid_loader.dataset)
    epoch_tr_loss.append(epoch_train_loss)
    epoch_vl_loss.append(epoch_val_loss)
    epoch_tr_acc.append(epoch_train_acc)
    epoch_vl_acc.append(epoch_val_acc)
    print(f'Epoch {epoch+1}')
    print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
    print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')

    if epoch_val_loss <= valid_loss_min:
        torch.save(model.state_dict(), 'textClassification.pt')
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
        valid_loss_min = epoch_val_loss
    print(25*'==')


In [ ]:
import matplotlib.pyplot as plt

# ... (Your existing code)

# After the training loop
plt.figure(figsize=(10, 5))
plt.plot(epoch_tr_loss, label='Training Loss')
plt.plot(epoch_vl_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.load_state_dict(torch.load('textClassification.pt'))
torch.save(model.state_dict(), '/content/drive/MyDrive/textClassification.pt')


In [ ]:

def predict_text(text,threshold_facotor=0.1):
        #each word is converted to integer using pretrained vocabulary vocab
        word_seq = np.array([vocab[preprocess_string(word)] for word in text.split()
                         if preprocess_string(word) in vocab.keys()])
        word_seq = np.expand_dims(word_seq,axis=0)
        pad =  torch.from_numpy(padding_(word_seq,500))
        inputs = pad.to(device)
        batch_size = inputs.shape[0]
        h = model.init_hidden(batch_size)
        h = tuple([each.data for each in h])
        output, h = model(inputs, h)

        # Assuming 'labels' are your true labels (ground truth)
        true_labels = y_t

    # Assuming 'output' is your predicted probabilities
        predicted_probs = output.cpu().detach().numpy()




        ##return(output.item())
       # print(output.item())
      #  predicted_class= 1 if tput.item()>5.475988018588396e-06= else 0
        return output.item()

In [ ]:

true_labels=[]
predicted_labels=[]
predicted_labelss=[]

for i in range(len(x)):
    pro=predict_text(x[i][1:-1])
    predicted_labels.append(pro)
    true_labels.append(y_t[i])

    #status = '"AI"' if pro==1 else '"Human"'
    #pro = (1 - pro) if status == '"Human"' else pro

     # Calculate ROC curve
pr, tpr, thresholds = roc_curve(y__t, predicted_labels)

    # Calculate AUC
auc_value = roc_auc_score(y__t, predicted_labels)

        # Plot ROC curve
plt.figure()
plt.plot(pr, tpr, color='darkorange', lw=2, label=f'AUC = {auc_value:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc='lower right')
plt.show()
        # Choose threshold based on the ROC curve
best_threshold_index = np.argmax(tpr - pr)
best_threshold = thresholds[best_threshold_index]



In [ ]:

for pro in predicted_labels:
    # Apply the threshold
    predicted_class = 1 if pro > best_threshold else 0
    status = '"AI"' if predicted_class==1 else '"Human"'
    predicted_labelss.append(status)


print(true_labels)
print('='*70)
print(predicted_labels)

In [ ]:

print(true_labels)
print('=' * 70)
print(predicted_labelss)

In [ ]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# Assuming true_labels and predicted_labels are your ground truth and predicted labels, respectively
accuracy = accuracy_score(true_labels, predicted_labelss) *100
precision = precision_score(true_labels, predicted_labelss, pos_label='"AI"')  # Assuming "AI" is the positive class
recall = recall_score(true_labels, predicted_labelss, pos_label='"AI"')  # Assuming "AI" is the positive class
f1 = f1_score(true_labels, predicted_labelss, pos_label='"AI"')  # Assuming "AI" is the positive class
roc_auc = roc_auc_score(true_labels, predicted_labels)
confusion = confusion_matrix(true_labels, predicted_labelss, labels=['"Human"', '"AI"'])
report = classification_report(true_labels, predicted_labelss)


print("Accuracy:", accuracy)


print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("ROC AUC:`", roc_auc)
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", report)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/textClassification.pt'))
model.eval()


new_df = pd.read_csv("drive/MyDrive/test.csv")

new_X, new_y = new_df['text'].values, new_df['label'].values
n_y=new_y.copy()
new_x_pad,new_y,new_x_pad,new_y,vocab = tockenize(new_X,new_y,new_X,new_y)
new__y=new_y.copy()

new_X_bigrams, bigram_counter_train = generate_bigrams(new_X)

new_X_bigrams_pad = padding_(new_X_bigrams, 500)

new_train_data = TensorDataset(torch.from_numpy(new_x_pad), torch.from_numpy(new_y))

new_train_loader = DataLoader(new_train_data, shuffle=True , batch_size=batch_size)

true_label=[]
predicted_label=[]
predicted_labels=[]

for i in range(len(new_X)):
    pro=predict_text(new_X[i][1:-1])
    predicted_label.append(pro)
    true_label.append(n_y[i])




     # Calculate ROC curve
pr, tpr, thresholds = roc_curve(new__y, predicted_label)

    # Calculate AUC
auc_value = roc_auc_score(new__y, predicted_label)

        # Plot ROC curve
plt.figure()
plt.plot(pr, tpr, color='darkorange', lw=2, label=f'AUC = {auc_value:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc='lower right')
plt.show()


for pro in predicted_label:
    # Apply the threshold
    predicted_class = 1 if pro > best_threshold else 0
    status = '"AI"' if predicted_class==1 else '"Human"'
    predicted_labels.append(status)

accuracy = accuracy_score(true_label, predicted_labels) *100
print(accuracy)
print(n_y)
print(predicted_labels)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report


model.load_state_dict(torch.load('/content/drive/MyDrive/textClassification.pt'))
model.eval()


new_df = pd.read_csv("drive/MyDrive/test_new.csv")

new_X, new_y = new_df['text'].values, new_df['label'].values
n_y=new_y.copy()
new_x_pad,new_y,new_x_pad,new_y,vocab = tockenize(new_X,new_y,new_X,new_y)
new__y=new_y.copy()

new_X_bigrams, bigram_counter_train = generate_bigrams(new_X)

new_X_bigrams_pad = padding_(new_X_bigrams, 500)

new_train_data = TensorDataset(torch.from_numpy(new_x_pad), torch.from_numpy(new_y))

new_train_loader = DataLoader(new_train_data, shuffle=True , batch_size=batch_size)

true_label=[]
predicted_label=[]
predicted_labels=[]
pred=[]

for i in range(len(new_X)):
    pro=predict_text(new_X[i][1:-1])
    predicted_label.append(pro)
    true_label.append(n_y[i])





     # Calculate ROC curve
pr, tpr, thresholds = roc_curve(new__y, predicted_label)

    # Calculate AUC
auc_value = roc_auc_score(new__y, predicted_label)

        # Plot ROC curve
plt.figure()
plt.plot(pr, tpr, color='darkorange', lw=2, label=f'AUC = {auc_value:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc='lower right')
plt.show()


for pro in predicted_label:
    # Apply the threshold
    predicted_class = 1 if pro > best_threshold else 0
    pred.append(predicted_class)
    status = '"AI"' if predicted_class==1 else '"Human"'
    predicted_labels.append(status)

np.savetxt('/content/drive/MyDrive/pred_LSTM.csv', pred, delimiter=',',  fmt='%.2f')


accuracy = accuracy_score(true_label, predicted_labels) *100
# Calculate accuracy
accuracy = accuracy_score(true_label, predicted_labels)
precision_ai = precision_score(true_label, predicted_labels, pos_label='"AI"')  # Assuming "AI" is the positive class
precision_human = precision_score(true_label, predicted_labels, pos_label='"Human"')  # Assuming "AI" is the positive class

recall_ai = recall_score(true_label, predicted_labels, pos_label='"AI"')  # Assuming "AI" is the positive class
recall_human = recall_score(true_label, predicted_labels, pos_label='"Human"')  # Assuming "AI" is the positive class

f1_ai = f1_score(true_label, predicted_labels, pos_label='"AI"')  # Assuming "AI" is the positive class
f1_human = f1_score(true_label, predicted_labels, pos_label='"Human"')  # Assuming "AI" is the positive class

conf_matrix = confusion_matrix(true_label, predicted_labels)


print("Accuracy:", accuracy)
print("Precision:", precision_ai)
print("Precision:", precision_human)

print("Recall:", recall_ai)
print("Recall:", recall_human)

print("F1-Score:", f1_ai)
print("F1-Score:", f1_human)
#print(n_y)
#print(predicted_labels)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data (replace with your actual values)
ai_values = [precision_ai,recall_ai,f1_ai]
human_values = [precision_human,recall_human,f1_human]

# Metrics names
metrics = [ 'Precision', 'Recall', 'F1-Score']

# Number of metrics
num_metrics = len(metrics)

# Create an array of indices for each metric
indices = np.arange(num_metrics)

# Bar width
bar_width = 0.1

# Plotting
fig, ax = plt.subplots(figsize=(6, 4))

# Plot AI values
ax.bar(indices, ai_values, bar_width, label='AI', color='blue',alpha=0.5)

# Plot Human values
ax.bar(indices + bar_width, human_values, bar_width, label='Human', color='orange')

# Customize the plot
# Make the y-axis tick labels darker and bigger
ax.tick_params(axis='both', labelsize=15, labelcolor='black')

# Make the axis labels (x and y) bold
ax.xaxis.label.set_fontweight('bold')
ax.yaxis.label.set_fontweight('bold')
ax.set_xticks(indices + bar_width / 2)

ax.set_xticklabels(metrics)
ax.legend(fontsize=9)
ax.set_ylabel('Metric Values')
#ax.set_title('Comparison of AI and Human Performance Metrics')

# Show the plot
plt.show()

In [ ]:
# Extract values from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Plotting
labels = ['True Negative', 'False Positive', 'False Negative', 'True Positive']
values = [tn, fp, fn, tp]

fig, ax = plt.subplots()
bars = ax.bar(labels, values, color=['pink', 'yellow', 'orange', 'blue'],alpha=0.8,width=0.3)

# Add labels and title
ax.set_ylabel('Count')
ax.set_title('Confusion Matrix')

# Add value annotations on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')
ax.tick_params(axis='y', labelsize=20, labelcolor='black')
ax.set_xticklabels(labels, fontweight='bold')

plt.show()